<a href="https://colab.research.google.com/github/OlgaAcosta/Project2_DataAnalytics_Henry/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hago la conexión con Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Descargo la librería requests
!pip install requests

# Importo las librerías

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import requests
import io


# **E T L**

# **I. EXTRACCIÓN DE DATOS**

Obtengo los datos de la página https://datosabiertos.enacom.gob.ar/dashboards/20000/acceso-a-internet/ por medio de su API

In [ ]:
# Creo una lista con los guids de cada dataset:
guids = ['PENET-DE-INTER-FIJO-57760', 'PENET-POR-HOGAR-NACIO-DEL', 'TOTAL-NACIO-DE-ACCES-48866',
         'BANDA-ANCHA-Y-BANDA-ANGOS','SERIE-TRIME-DE-ACCES-A','ACCES-A-INTER-FIJO-POR','VELOC-PROME-DE-BAJAD-DE',
         'DISTR-DE-LOS-ACCES-76145','ACCES-A-INTER-FIJO-23248','ACCES-A-INTER-FIJO-97624','INGRE-TRIME-POR-LA-PREST',
         'ACCES-A-INTER-FIJO-16249', 'ACCES-A-INTER-FIJO-62463','LISTA-DE-LOCAL-CON-CONEC','CONEC-AL-SERVI-DE-INTER']

# Guardo la key provista para la API:
auth_key= "jj8swcy26oKsuw7GJ4Ucox7ACpDj87Poa1B4rlDj"

In [ ]:

# Itero sobre cada guid
for guid in guids:
    url = f'https://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/{guid}/data.csv/?auth_key={auth_key}'

    response = requests.get(url)

    if response.status_code == 200:
        result = response.content
        file_name = f'/content/drive/MyDrive/HENRY/PI_DA/Datasets_API/Internet/{guid}.csv'  # Nombre del archivo basado en el GUID
        with open(file_name, 'wb') as file:
            file.write(result)
        print(f'Archivo "{file_name}" guardado con éxito.')
    else:
        print(f'Error en la solicitud para GUID "{guid}":', response.status_code)

Error en la solicitud para GUID "PENET-DE-INTER-FIJO-57760": 500
Error en la solicitud para GUID "PENET-POR-HOGAR-NACIO-DEL": 500
Archivo "/content/drive/MyDrive/HENRY/PI_DA/Datasets_originales/Internet/TOTAL-NACIO-DE-ACCES-48866.csv" guardado con éxito.
Archivo "/content/drive/MyDrive/HENRY/PI_DA/Datasets_originales/Internet/BANDA-ANCHA-Y-BANDA-ANGOS.csv" guardado con éxito.
Error en la solicitud para GUID "SERIE-TRIME-DE-ACCES-A": 500
Error en la solicitud para GUID "ACCES-A-INTER-FIJO-POR": 500
Error en la solicitud para GUID "VELOC-PROME-DE-BAJAD-DE": 500
Error en la solicitud para GUID "DISTR-DE-LOS-ACCES-76145": 500
Archivo "/content/drive/MyDrive/HENRY/PI_DA/Datasets_originales/Internet/ACCES-A-INTER-FIJO-23248.csv" guardado con éxito.
Error en la solicitud para GUID "ACCES-A-INTER-FIJO-97624": 500
Archivo "/content/drive/MyDrive/HENRY/PI_DA/Datasets_originales/Internet/INGRE-TRIME-POR-LA-PREST.csv" guardado con éxito.
Archivo "/content/drive/MyDrive/HENRY/PI_DA/Datasets_origina

* Se pudo obtener 7 dataframes ya que los restantes devolvieron un error.


* Al no poder descargar todos los datasets restantes de la página https://datosabiertos.enacom.gob.ar/home (ya que no están disponibles en ningún formato) y con el fin de no cruzar datos de periodos distintos (ya que los obtenidos por la API están actualizados), opté por utilizar los datasets que provee Henry, los cuales tienen data solo hasta el primer trimestre del 2022.

# **II. TRANSFORMACIÓN**

# I.I DATASETS DE INTERNET

## **Aclaración**

Después de revisar a grosso modo el contenido de cada dataset por separado, he decidido orientar mi *objetivo general* al **alcance de la cobertura de red en Argentina** ya que me parece una necesidad importante de abordar, por lo que he seleccionado solo aquellos datasets que provean datos relacionados a este tema. Así, he descartado datasets relativos a otros aspectos y/o redundantes.

A partir de los datasets seleccionados realizaré el EDA correspondiente para, luego, establecer mi *objetivo principal* y propuesta para el negocio, y determinar los KPI´s a utilizar.

Empezaré los siguientes datasets correspondientes al servicio de internet en Argentina:

* Acceso_Internet_Fijo_por_rangos_velocidad_bajada_y_provincia

* Acceso_Internet_fijo_por_tecnología_y_provincia

* Accesos_banda_ancha_y_angosta_por_provincia

* Penetración_de _Internet fijo _(accesos _cada_100 hogares)

* Velocidad_media_bajada_Internet_fijo_por_provincia

Guardo los datasets en sus respectivos dataframes y especifico que los lea como *string* para que no haya error en la lectura de los datos, ya que contienen puntos.

In [7]:
acc_velocidad_rangos = pd.read_csv("/content/drive/MyDrive/HENRY/PI_DA/Datasets/Internet/Acceso_Internet_Fijo_por_rangos_velocidad_bajada_y_provincia.csv", sep=",", dtype= str)
acc_tecnologia_prov = pd.read_csv("/content/drive/MyDrive/HENRY/PI_DA/Datasets/Internet/Acceso_Internet_fijo_por_tecnología_y_provincia.csv", sep=",", dtype= str)
acc_banda_prov = pd.read_csv("/content/drive/MyDrive/HENRY/PI_DA/Datasets/Internet/Accesos_banda_ancha_y_angosta_por_provincia.csv", sep=",",dtype= str)
pen_cien_hogares = pd.read_csv("/content/drive/MyDrive/HENRY/PI_DA/Datasets/Internet/Penetración_de _Internet fijo _(accesos _cada_100 hogares).csv", sep=",",dtype= str)
velocidad_media_prov = pd.read_csv("/content/drive/MyDrive/HENRY/PI_DA/Datasets/Internet/Velocidad_media_bajada_Internet_fijo_por_provincia.csv", sep=",",dtype= str)


## **DF: acc_velocidad_rangos**

In [8]:
print(acc_velocidad_rangos.info())
acc_velocidad_rangos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Año                  792 non-null    object
 1   Trimestre            792 non-null    object
 2   Provincia            792 non-null    object
 3   HASTA 512 kbps       792 non-null    object
 4   + 512 Kbps - 1 Mbps  792 non-null    object
 5   + 1 Mbps - 6 Mbps    792 non-null    object
 6   + 6 Mbps - 10 Mbps   792 non-null    object
 7   + 10 Mbps - 20 Mbps  792 non-null    object
 8   + 20 Mbps - 30 Mbps  792 non-null    object
 9   + 30 Mbps            792 non-null    object
 10  OTROS                786 non-null    object
 11  Total                792 non-null    object
dtypes: object(12)
memory usage: 74.4+ KB
None


,Año,Trimestre,Provincia,HASTA 512 kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,OTROS,Total
0,2022,1,Buenos Aires,31.591,30.056,313.382,321.756,290.127,161.183,3.381.049,26.280,4.555.424
1,2022,1,Capital Federal,527,5.575,39.918,77.390,61.053,43.289,1.188.072,1.717,1.417.541
2,2022,1,Catamarca,71,456,4.386,7.009,8.773,3.761,35.715,2.207,62.378
3,2022,1,Chaco,461,1.099,16.888,21.235,20.898,13.012,62.946,7.580,144.119
4,2022,1,Chubut,113,1.677,61.369,31.856,33.080,13.871,14.055,15.607,171.628


In [9]:
type(acc_velocidad_rangos["OTROS"][0])

str

* El tipo de dato de las columnas *Año* y *Trimestre* deben ser cambiadas a *integer*.
* Quito los puntos de todas las columnas numéricas para luego poder convertir los datos al tipo float :

In [10]:
# Hago una lista con las columnas necesarias:
columnas= ['HASTA 512 kbps','+ 512 Kbps - 1 Mbps','+ 1 Mbps - 6 Mbps','+ 6 Mbps - 10 Mbps',
           '+ 10 Mbps - 20 Mbps','+ 20 Mbps - 30 Mbps','+ 30 Mbps', 'OTROS' , 'Total' ]

# Reemplazo los "." por "":
acc_velocidad_rangos[columnas] = acc_velocidad_rangos[columnas].apply(lambda x: x.str.replace('.', ''))

# Convierto a float:
acc_velocidad_rangos[columnas] = acc_velocidad_rangos[columnas].astype(float)

# Convierto a integer:
acc_velocidad_rangos[["Año", "Trimestre"]] = acc_velocidad_rangos[["Año", "Trimestre"]].astype(int)

# Reviso:
print(acc_velocidad_rangos.info())
acc_velocidad_rangos.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Año                  792 non-null    int64  
 1   Trimestre            792 non-null    int64  
 2   Provincia            792 non-null    object 
 3   HASTA 512 kbps       792 non-null    float64
 4   + 512 Kbps - 1 Mbps  792 non-null    float64
 5   + 1 Mbps - 6 Mbps    792 non-null    float64
 6   + 6 Mbps - 10 Mbps   792 non-null    float64
 7   + 10 Mbps - 20 Mbps  792 non-null    float64
 8   + 20 Mbps - 30 Mbps  792 non-null    float64
 9   + 30 Mbps            792 non-null    float64
 10  OTROS                786 non-null    float64
 11  Total                792 non-null    float64
dtypes: float64(9), int64(2), object(1)
memory usage: 74.4+ KB
None


<ipython-input-10-6aca20baddfd>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  acc_velocidad_rangos[columnas] = acc_velocidad_rangos[columnas].apply(lambda x: x.str.replace('.', ''))


,Año,Trimestre,Provincia,HASTA 512 kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,OTROS,Total
0,2022,1,Buenos Aires,31591.0,30056.0,313382.0,321756.0,290127.0,161183.0,3381049.0,26280.0,4555424.0
1,2022,1,Capital Federal,527.0,5575.0,39918.0,77390.0,61053.0,43289.0,1188072.0,1717.0,1417541.0


Ya están normalizados los datos necesarios.
Ahora, reviso los otros dataframes para confirmar el mismo problema:


##**DF : acc_tecnologia_prov**

In [11]:
print(acc_tecnologia_prov.info())
acc_tecnologia_prov.head(2)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Año           793 non-null    object
 1   Trimestre     793 non-null    object
 2   Provincia     792 non-null    object
 3   ADSL          792 non-null    object
 4   Cablemodem    792 non-null    object
 5   Fibra óptica  792 non-null    object
 6   Wireless      792 non-null    object
 7   Otros         792 non-null    object
 8   Total         792 non-null    object
dtypes: object(9)
memory usage: 55.9+ KB
None


,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2022,1,Buenos Aires,392.045,2.728.865,1.242.121,125.521,66.872,4.555.424
1,2022,1,Capital Federal,102.290,1.217.225,62.308,5.784,29.934,1.417.541


* Procedo con las mismas transformaciones: quitar los "." y convertir a float
* Encontré que hay un registro "- 0" que no permite la conversión a float; por lo que lo reemplazaré por "0" ya que es muy probable que sea un error humano.

In [12]:
# Hago una lista con las columnas necesarias:
columnas= ['ADSL','Cablemodem','Fibra óptica','Wireless',
           'Otros','Total']

# Reemplazo los "." por "":
acc_tecnologia_prov[columnas] = acc_tecnologia_prov[columnas].apply(lambda x: x.str.replace('.', ''))

# Reemplazo el "-0" por "0":
acc_tecnologia_prov[columnas] = acc_tecnologia_prov[columnas].apply(lambda x: x.str.replace('- 0', '0'))

# Convierto a float:
acc_tecnologia_prov[columnas] = acc_tecnologia_prov[columnas].astype(float)

# Reviso:
print(acc_tecnologia_prov.info())
acc_tecnologia_prov.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Año           793 non-null    object 
 1   Trimestre     793 non-null    object 
 2   Provincia     792 non-null    object 
 3   ADSL          792 non-null    float64
 4   Cablemodem    792 non-null    float64
 5   Fibra óptica  792 non-null    float64
 6   Wireless      792 non-null    float64
 7   Otros         792 non-null    float64
 8   Total         792 non-null    float64
dtypes: float64(6), object(3)
memory usage: 55.9+ KB
None


<ipython-input-12-e5e99190ed9c>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  acc_tecnologia_prov[columnas] = acc_tecnologia_prov[columnas].apply(lambda x: x.str.replace('.', ''))


,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2022,1,Buenos Aires,392045.0,2728865.0,1242121.0,125521.0,66872.0,4555424.0
1,2022,1,Capital Federal,102290.0,1217225.0,62308.0,5784.0,29934.0,1417541.0


* Ahora, reviso los valores únicos de cada columna categórica:

In [13]:
print(acc_tecnologia_prov["Año"].value_counts() )
print(acc_tecnologia_prov["Trimestre"].value_counts())
print(acc_tecnologia_prov["Provincia"].value_counts())


2021      96
2020      96
2018      96
2017      96
2016      96
2015      96
2014      96
2019 *    72
2022      24
2019      24
*          1
Name: Año, dtype: int64
1                                                                                                                                     192
4                                                                                                                                     192
3                                                                                                                                     168
2                                                                                                                                     168
3 *                                                                                                                                    24
2 *                                                                                                                                    24
1 *  

* Esto indica que en la columna *Año* del df *acc_tecnologia_prov* hay un valor *2019 **; ubico los valores para descubrir las causas:

In [14]:
acc_tecnologia_prov[acc_tecnologia_prov['Año'] == '2019 *']

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
240,2019 *,3 *,Buenos Aires,1277249.0,2144956.0,425386.0,85572.0,27070.0,3960233.0
241,2019 *,3 *,Capital Federal,300288.0,1080973.0,24900.0,3895.0,31155.0,1441211.0
242,2019 *,3 *,Catamarca,24349.0,5543.0,13.0,244.0,2251.0,32400.0
243,2019 *,3 *,Chaco,50416.0,49156.0,8063.0,5224.0,1592.0,114451.0
244,2019 *,3 *,Chubut,37298.0,27090.0,1917.0,15149.0,7551.0,89005.0
...,...,...,...,...,...,...,...,...,...
307,2019 *,1 *,Santa Cruz,10659.0,4615.0,382.0,6514.0,2735.0,24905.0
308,2019 *,1 *,Santa Fe,340038.0,340344.0,15621.0,23215.0,17860.0,737078.0
309,2019 *,1 *,Santiago Del Estero,25337.0,61547.0,23.0,2847.0,99.0,89853.0
310,2019 *,1 *,Tierra Del Fuego,19789.0,23549.0,1623.0,0.0,1486.0,46447.0


* Hay asteriscos también en la columna trimestre; reviso, entonces, los valores únicos de esta columnas:

In [15]:
acc_tecnologia_prov['Trimestre'].value_counts()

1                                                                                                                                     192
4                                                                                                                                     192
3                                                                                                                                     168
2                                                                                                                                     168
3 *                                                                                                                                    24
2 *                                                                                                                                    24
1 *                                                                                                                                    24
Los datos provinciales no coincide

* Encuentro que hay un registro anómalo; lo ubico:

In [16]:
acc_tecnologia_prov[acc_tecnologia_prov['Trimestre'] == 'Los datos provinciales no coinciden a nivel nacional, ya que se rincorporó información que no contien apertuta a nivel geográfico.']

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
792,*,Los datos provinciales no coinciden a nivel na...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Este registro parece indicar que todos los datos acompañados de un "*" no coinciden a nivel nacional, sin embargo, al revisar los otros datasets, encuentro que sí hay coincidencia.
* Al no tener mayor información, procedo a reemplazar esos asteriscos:

In [17]:
# Reemplazo los asteriscos por "":
acc_tecnologia_prov['Trimestre'] = acc_tecnologia_prov['Trimestre'].apply(lambda x: x.replace(' *', ''))
acc_tecnologia_prov['Año'] = acc_tecnologia_prov['Año'].apply(lambda x: x.replace(' *', ''))

# Reviso:
print(acc_tecnologia_prov['Trimestre'].value_counts())
print(acc_tecnologia_prov['Año'].value_counts())

1                                                                                                                                     216
4                                                                                                                                     192
3                                                                                                                                     192
2                                                                                                                                     192
Los datos provinciales no coinciden a nivel nacional, ya que se rincorporó información que no contien apertuta a nivel geográfico.      1
Name: Trimestre, dtype: int64
2021    96
2020    96
2019    96
2018    96
2017    96
2016    96
2015    96
2014    96
2022    24
*        1
Name: Año, dtype: int64


##**DF : acc_banda_prov**

In [18]:
print(acc_banda_prov.info())
acc_banda_prov.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Año               792 non-null    object
 1   Trimestre         792 non-null    object
 2   Provincia         792 non-null    object
 3   Banda ancha fija  792 non-null    object
 4   Dial up           790 non-null    object
 5   Total             792 non-null    object
dtypes: object(6)
memory usage: 37.2+ KB
None


,Año,Trimestre,Provincia,Banda ancha fija,Dial up,Total
0,2022,1,Buenos Aires,4.549.131,6.293,4.555.424
1,2022,1,Capital Federal,1.415.378,2.163,1.417.541


* Procedo con las mismas transformaciones: quitar los "." y convertir a float
* Encontré que hay un registro "- 0" que no permite la conversión a float; por lo que lo reemplazaré por "0" ya que es muy probable que sea un error humano.

In [19]:
# Hago una lista con las columnas necesarias:
columnas= ['Banda ancha fija','Dial up','Total']

# Reemplazo los "." por "":
acc_banda_prov[columnas] = acc_banda_prov[columnas].apply(lambda x: x.str.replace('.', ''))

# Reemplazo el "-0" por "0":
acc_banda_prov[columnas] = acc_banda_prov[columnas].apply(lambda x: x.str.replace('- 0', '0'))

# Convierto a float:
acc_banda_prov[columnas] = acc_banda_prov[columnas].astype(float)

# Reviso:
print(acc_banda_prov.info())
acc_banda_prov.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Año               792 non-null    object 
 1   Trimestre         792 non-null    object 
 2   Provincia         792 non-null    object 
 3   Banda ancha fija  792 non-null    float64
 4   Dial up           790 non-null    float64
 5   Total             792 non-null    float64
dtypes: float64(3), object(3)
memory usage: 37.2+ KB
None


<ipython-input-19-d26d5fc7a591>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  acc_banda_prov[columnas] = acc_banda_prov[columnas].apply(lambda x: x.str.replace('.', ''))


,Año,Trimestre,Provincia,Banda ancha fija,Dial up,Total
0,2022,1,Buenos Aires,4549131.0,6293.0,4555424.0
1,2022,1,Capital Federal,1415378.0,2163.0,1417541.0


##**DF : pen_cien_hogares**

In [20]:
print(pen_cien_hogares.info())
pen_cien_hogares.head(2)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Año                           792 non-null    object
 1   Trimestre                     792 non-null    object
 2   Provincia                     792 non-null    object
 3   Accesos por cada 100 hogares  792 non-null    object
dtypes: object(4)
memory usage: 24.9+ KB
None


,Año,Trimestre,Provincia,Accesos por cada 100 hogares
0,2022,1,Buenos Aires,"76,08"
1,2022,1,Capital Federal,"111,8"


* Reemplazo las "," y convierto a float
* Encontré que hay un registro "- 0" que no permite la conversión a float; por lo que lo reemplazaré por "0" ya que es muy probable que sea un error humano.

In [21]:
# Hago una lista con las columnas necesarias:
columna= ['Accesos por cada 100 hogares']

# Reemplazo los "." por "":
pen_cien_hogares[columna] = pen_cien_hogares[columna].apply(lambda x: x.str.replace(',', ''))

# Convierto a float:
pen_cien_hogares[columna] = pen_cien_hogares[columna].astype(float)

# Reviso:
print(pen_cien_hogares.info())
pen_cien_hogares.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Año                           792 non-null    object 
 1   Trimestre                     792 non-null    object 
 2   Provincia                     792 non-null    object 
 3   Accesos por cada 100 hogares  792 non-null    float64
dtypes: float64(1), object(3)
memory usage: 24.9+ KB
None


,Año,Trimestre,Provincia,Accesos por cada 100 hogares
0,2022,1,Buenos Aires,7608.0
1,2022,1,Capital Federal,1118.0


##**DF : velocidad_media_prov**

In [22]:
print(velocidad_media_prov.info())
velocidad_media_prov.tail(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Año                     792 non-null    object
 1   Trimestre               792 non-null    object
 2   Provincia               792 non-null    object
 3   Mbps (Media de bajada)  792 non-null    object
dtypes: object(4)
memory usage: 24.9+ KB
None


,Año,Trimestre,Provincia,Mbps (Media de bajada)
782,2014,1,Neuquén,4
783,2014,1,Río Negro,3
784,2014,1,Salta,3
785,2014,1,San Juan,3
786,2014,1,San Luis,4
787,2014,1,Santa Cruz,3
788,2014,1,Santa Fe,3
789,2014,1,Santiago Del Estero,3
790,2014,1,Tierra Del Fuego,3
791,2014,1,Tucumán,3


* El dataframe *velocidad_media_prov* no contiene puntos ni comas en las columnas numéricas, así que solo cambio el tipo de dato a float:

In [23]:
# Convierto a float:
velocidad_media_prov["Mbps (Media de bajada)"] = velocidad_media_prov["Mbps (Media de bajada)"].astype(float)

# Reviso:
print(velocidad_media_prov.info())
velocidad_media_prov.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Año                     792 non-null    object 
 1   Trimestre               792 non-null    object 
 2   Provincia               792 non-null    object 
 3   Mbps (Media de bajada)  792 non-null    float64
dtypes: float64(1), object(3)
memory usage: 24.9+ KB
None


,Año,Trimestre,Provincia,Mbps (Media de bajada)
0,2022,1,Buenos Aires,63.0
1,2022,1,Capital Federal,88.0


**¡LISTO!** Ya tengo las columnas normalizadas.

* De acuerdo a la información de cada dataframe, observo que todos ellos comparten tres columnas: *Año*, *Trimestre* y *Provincia*.
* De acuerdo a lo anterior, puedo unir los df a partir de estos tres campos a fin de facilitar el análisis y manejo de los datos.

In [24]:
# Defino las columnas para la unión:

col_merge= ["Año", "Trimestre", "Provincia"]

# MERGE

merged_df = pd.merge(acc_tecnologia_prov, acc_banda_prov.drop("Total", axis=1), on=col_merge) #, how='outer')

merged_df2 = pd.merge(merged_df, pen_cien_hogares, on=col_merge) #, how='outer' )

internet_prov = pd.merge(merged_df2, velocidad_media_prov, on=col_merge) #, how='outer')


In [25]:
print(internet_prov.info())
internet_prov.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 792 entries, 0 to 791
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Año                           792 non-null    object 
 1   Trimestre                     792 non-null    object 
 2   Provincia                     792 non-null    object 
 3   ADSL                          792 non-null    float64
 4   Cablemodem                    792 non-null    float64
 5   Fibra óptica                  792 non-null    float64
 6   Wireless                      792 non-null    float64
 7   Otros                         792 non-null    float64
 8   Total                         792 non-null    float64
 9   Banda ancha fija              792 non-null    float64
 10  Dial up                       790 non-null    float64
 11  Accesos por cada 100 hogares  792 non-null    float64
 12  Mbps (Media de bajada)        792 non-null    float64
dtypes: fl

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Banda ancha fija,Dial up,Accesos por cada 100 hogares,Mbps (Media de bajada)
0,2022,1,Buenos Aires,392045.0,2728865.0,1242121.0,125521.0,66872.0,4555424.0,4549131.0,6293.0,7608.0,63.0
1,2022,1,Capital Federal,102290.0,1217225.0,62308.0,5784.0,29934.0,1417541.0,1415378.0,2163.0,1118.0,88.0
2,2022,1,Catamarca,14181.0,15517.0,29118.0,1328.0,2234.0,62378.0,62377.0,1.0,5821.0,51.0


* Reviso los nulos del nuevo dataframe:

In [26]:
# Nulos:
internet_prov.isnull().sum()


Año                             0
Trimestre                       0
Provincia                       0
ADSL                            0
Cablemodem                      0
Fibra óptica                    0
Wireless                        0
Otros                           0
Total                           0
Banda ancha fija                0
Dial up                         2
Accesos por cada 100 hogares    0
Mbps (Media de bajada)          0
dtype: int64

* Ubico nulos:

In [27]:
internet_prov[internet_prov["Dial up"].isnull()]

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Banda ancha fija,Dial up,Accesos por cada 100 hogares,Mbps (Media de bajada)
368,2018,2,Formosa,23543.0,7311.0,18.0,13625.0,57.0,44554.0,44554.0,NaN,2943.0,9.0
373,2018,2,Misiones,57265.0,47923.0,512.0,391.0,56.0,106147.0,106147.0,NaN,3163.0,12.0


* Los nulos en la columna *Dial up* son en realidad *0*´s, ya que la suma de *Banda ancha fija*  y *Dial up* debe dar el *Total*.
* Por lo anterior, imputo los nulos por *0*:

In [28]:
internet_prov["Dial up"].fillna("0", inplace=True)

# Reviso:
internet_prov[internet_prov["Dial up"].isnull()]

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Banda ancha fija,Dial up,Accesos por cada 100 hogares,Mbps (Media de bajada)


* Reviso duplicados del nuevo dataframe:

In [29]:
# Duplicados:
internet_prov.duplicated().sum()

0

* No hay duplicados.
* Por último, cambio el tipo de datos de las columnas *Año* y *Trimestre*:

In [30]:
internet_prov[["Año", "Trimestre"]] = internet_prov[["Año", "Trimestre"]].astype(int)

# Reviso:
internet_prov.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 792 entries, 0 to 791
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Año                           792 non-null    int64  
 1   Trimestre                     792 non-null    int64  
 2   Provincia                     792 non-null    object 
 3   ADSL                          792 non-null    float64
 4   Cablemodem                    792 non-null    float64
 5   Fibra óptica                  792 non-null    float64
 6   Wireless                      792 non-null    float64
 7   Otros                         792 non-null    float64
 8   Total                         792 non-null    float64
 9   Banda ancha fija              792 non-null    float64
 10  Dial up                       792 non-null    object 
 11  Accesos por cada 100 hogares  792 non-null    float64
 12  Mbps (Media de bajada)        792 non-null    float64
dtypes: fl

## Continuaré con los siguientes datasets correspondientes al servicio de internet en Argentina:
* Accesos_Internet_fijo_por_tecnología_y_localidad.
* Conectividad_al_servicio_Internet

Guardo los datasets en sus respectivos dataframes y especifico que los lea como *string* para que no haya error en la lectura de los datos, ya que contienen puntos.

In [31]:
# Importo los csv:
acc_localidad= pd.read_csv("/content/drive/MyDrive/HENRY/PI_DA/Datasets/Internet/Accesos_Internet_fijo_por_tecnología_y_localidad.csv", sep=",", dtype= str)
conectividad_localidad= pd.read_csv("/content/drive/MyDrive/HENRY/PI_DA/Datasets/Internet/Conectividad_al_servicio_Internet.csv", sep=",", dtype= str)

## **DF: acc_localidad**

In [32]:
print(acc_localidad.info())
acc_localidad.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3127 entries, 0 to 3126
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Provincia      3127 non-null   object
 1   Partido        3127 non-null   object
 2   Localidad      3127 non-null   object
 3   Link Indec     3127 non-null   object
 4   ADSL           3127 non-null   object
 5   CABLEMODEM     3127 non-null   object
 6   DIAL UP        3127 non-null   object
 7   FIBRA OPTICA   3127 non-null   object
 8   OTROS          3127 non-null   object
 9   SATELITAL      3127 non-null   object
 10  WIMAX          3127 non-null   object
 11  WIRELESS       3127 non-null   object
 12  Total general  3127 non-null   object
 13  Unnamed: 13    0 non-null      object
dtypes: object(14)
memory usage: 342.1+ KB
None


,Provincia,Partido,Localidad,Link Indec,ADSL,CABLEMODEM,DIAL UP,FIBRA OPTICA,OTROS,SATELITAL,WIMAX,WIRELESS,Total general,Unnamed: 13
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,914,4.704,- 0,5,- 0,975,- 0,664,7.262,NaN
1,BUENOS AIRES,25 de Mayo,Del Valle,6854020,181,- 0,- 0,- 0,10,1,- 0,- 0,192,NaN
2,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,6854040,- 0,- 0,- 0,- 0,- 0,- 0,- 0,181,181,NaN
3,BUENOS AIRES,25 de Mayo,Norberto de la Riestra,6854060,- 0,782,- 0,- 0,18,- 0,- 0,327,1.127,NaN
4,BUENOS AIRES,25 de Mayo,Lucas Monteverde,6854050,- 0,- 0,- 0,- 0,- 0,- 0,- 0,6,6,NaN


* Reviso la columna *Link Indec*:

In [33]:
acc_localidad["Link Indec"].value_counts()

Sin datos    196
6371010       26
6441030       26
6840010       15
6427010       15
            ... 
14140290       1
14140300       1
14140310       1
14140320       1
90119030       1
Name: Link Indec, Length: 2698, dtype: int64

* Este dataframe no presenta nulos.
* La columna *Link Indec* tiene repetidos y valores sin dato. No me provee información relevante.
* Observo que hay valores *-0* y una columna vacía.
* Reemplazo los puntos y los *-0* , elimino la columna vacía y *Link Indec*:

In [34]:
# Hago una lista con las columnas necesarias:
columnas= ['ADSL','CABLEMODEM','DIAL UP','FIBRA OPTICA',
           'OTROS','SATELITAL','WIMAX', 'WIRELESS' , 'Total general']

# Reemplazo los "." por "":
acc_localidad[columnas] = acc_localidad[columnas].apply(lambda x: x.str.replace('.', ''))

# Reemplazo el "-0" por "0":
acc_localidad[columnas] = acc_localidad[columnas].apply(lambda x: x.str.replace('- 0', '0'))

# Convierto a float:
acc_localidad[columnas] = acc_localidad[columnas].astype(float)

# Elimino la columna vacía:
acc_localidad.drop(["Unnamed: 13", "Link Indec"], axis=1, inplace=True)

# Reviso:
print(acc_localidad.info())
acc_localidad.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3127 entries, 0 to 3126
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Provincia      3127 non-null   object 
 1   Partido        3127 non-null   object 
 2   Localidad      3127 non-null   object 
 3   ADSL           3127 non-null   float64
 4   CABLEMODEM     3127 non-null   float64
 5   DIAL UP        3127 non-null   float64
 6   FIBRA OPTICA   3127 non-null   float64
 7   OTROS          3127 non-null   float64
 8   SATELITAL      3127 non-null   float64
 9   WIMAX          3127 non-null   float64
 10  WIRELESS       3127 non-null   float64
 11  Total general  3127 non-null   float64
dtypes: float64(9), object(3)
memory usage: 293.3+ KB
None


<ipython-input-34-6bd1d0317c7e>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  acc_localidad[columnas] = acc_localidad[columnas].apply(lambda x: x.str.replace('.', ''))


,Provincia,Partido,Localidad,ADSL,CABLEMODEM,DIAL UP,FIBRA OPTICA,OTROS,SATELITAL,WIMAX,WIRELESS,Total general
0,BUENOS AIRES,25 de Mayo,25 de Mayo,914.0,4704.0,0.0,5.0,0.0,975.0,0.0,664.0,7262.0
1,BUENOS AIRES,25 de Mayo,Del Valle,181.0,0.0,0.0,0.0,10.0,1.0,0.0,0.0,192.0


## **DF: conectividad_localidad**

In [35]:
print(conectividad_localidad.info())
conectividad_localidad.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4312 entries, 0 to 4311
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Provincia      4312 non-null   object
 1   Partido        4312 non-null   object
 2   Localidad      4312 non-null   object
 3   Poblacion      4312 non-null   object
 4   ADSL           4312 non-null   object
 5   CABLEMODEM     4312 non-null   object
 6   DIALUP         4312 non-null   object
 7   FIBRAOPTICA    4312 non-null   object
 8   SATELITAL      4312 non-null   object
 9   WIRELESS       4312 non-null   object
 10  TELEFONIAFIJA  4312 non-null   object
 11  3G             4312 non-null   object
 12  4G             4312 non-null   object
 13  link           3836 non-null   object
 14  Latitud        4312 non-null   object
 15  Longitud       4312 non-null   object
dtypes: object(16)
memory usage: 539.1+ KB
None


,Provincia,Partido,Localidad,Poblacion,ADSL,CABLEMODEM,DIALUP,FIBRAOPTICA,SATELITAL,WIRELESS,TELEFONIAFIJA,3G,4G,link,Latitud,Longitud
0,BUENOS AIRES,25 de Mayo,25 de Mayo,23408,SI,SI,--,SI,SI,SI,SI,SI,SI,6854100,-35.4339385892588,-60.1731209454056
1,BUENOS AIRES,25 de Mayo,Agustín Mosconi,280,--,--,--,--,--,--,SI,--,--,6854010,-35.7392169542026,-60.5590634621789
2,BUENOS AIRES,25 de Mayo,Del Valle,899,SI,--,--,--,SI,--,--,SI,SI,6854020,-35.8973453887603,-60.7316089512944
3,BUENOS AIRES,25 de Mayo,Ernestina,145,SI,--,--,--,--,--,--,--,--,6854030,-35.2702340519789,-59.5575034078873
4,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,547,--,--,--,--,--,SI,--,--,--,6854040,-35.1643973648093,-60.0813577815069


* Reviso la columna *link*:

In [36]:
conectividad_localidad["link"].value_counts()

2000010     48
6441030     31
6371010     27
6638040     17
6840010     15
            ..
86091005     1
86091010     1
86091020     1
86091030     1
38014080     1
Name: link, Length: 3523, dtype: int64

* Este dataframe no presenta nulos.
* La columna *link*  tiene repetidos y valores nulos. No me provee información relevante.
* Las columnas númericas están en tipo object. Cambio los tipos y elimino la columna *link*:


In [37]:
# Hago una lista con las columnas necesarias:
columnas= ['Poblacion','Latitud','Longitud']

# Convierto a float:
conectividad_localidad[columnas] = conectividad_localidad[columnas].astype(float)

# Elimino la columna vacía:
conectividad_localidad.drop("link", axis=1, inplace=True)

# Reviso:
print(conectividad_localidad.info())
conectividad_localidad.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4312 entries, 0 to 4311
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Provincia      4312 non-null   object 
 1   Partido        4312 non-null   object 
 2   Localidad      4312 non-null   object 
 3   Poblacion      4312 non-null   float64
 4   ADSL           4312 non-null   object 
 5   CABLEMODEM     4312 non-null   object 
 6   DIALUP         4312 non-null   object 
 7   FIBRAOPTICA    4312 non-null   object 
 8   SATELITAL      4312 non-null   object 
 9   WIRELESS       4312 non-null   object 
 10  TELEFONIAFIJA  4312 non-null   object 
 11  3G             4312 non-null   object 
 12  4G             4312 non-null   object 
 13  Latitud        4312 non-null   float64
 14  Longitud       4312 non-null   float64
dtypes: float64(3), object(12)
memory usage: 505.4+ KB
None


,Provincia,Partido,Localidad,Poblacion,ADSL,CABLEMODEM,DIALUP,FIBRAOPTICA,SATELITAL,WIRELESS,TELEFONIAFIJA,3G,4G,Latitud,Longitud
0,BUENOS AIRES,25 de Mayo,25 de Mayo,23408.0,SI,SI,--,SI,SI,SI,SI,SI,SI,-35.433939,-60.173121
1,BUENOS AIRES,25 de Mayo,Agustín Mosconi,280.0,--,--,--,--,--,--,SI,--,--,-35.739217,-60.559063


**¡LISTO!** Ya tengo las columnas necesarias normalizadas.

* De acuerdo a la información de cada dataframe, observo que ambos comparten varias columnas relativas a tecnologías : 'ADSL', 'CABLEMODEM', 'DIAL UP','FIBRA OPTICA', 'SATELITAL', 'WIRELESS'; sin embargo, el primero posee la cantidad de accesos y el segundo un valor de tipo booleano "SI". En este caso y en consecución de mi objetivo general planteado, creo que es de más valor quedarme con los datos de la cantidad, para poder compararlos y analizarlos.
* De acuerdo a lo anterior, uno los df a partir de los campos *Provincia*, *Partido* y *Localidad*, a fin de facilitar el análisis y manejo de los datos:

In [38]:
# Defino las columnas para la unión:
col_merge= ["Provincia", "Partido", "Localidad"]

# MERGE
internet_loc= pd.merge(acc_localidad, conectividad_localidad.drop(["ADSL", "CABLEMODEM","DIALUP", "FIBRAOPTICA", "SATELITAL", "WIRELESS"], axis=1), on =col_merge)

# Reviso:
print(internet_loc.info())
internet_loc.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2968 entries, 0 to 2967
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Provincia      2968 non-null   object 
 1   Partido        2968 non-null   object 
 2   Localidad      2968 non-null   object 
 3   ADSL           2968 non-null   float64
 4   CABLEMODEM     2968 non-null   float64
 5   DIAL UP        2968 non-null   float64
 6   FIBRA OPTICA   2968 non-null   float64
 7   OTROS          2968 non-null   float64
 8   SATELITAL      2968 non-null   float64
 9   WIMAX          2968 non-null   float64
 10  WIRELESS       2968 non-null   float64
 11  Total general  2968 non-null   float64
 12  Poblacion      2968 non-null   float64
 13  TELEFONIAFIJA  2968 non-null   object 
 14  3G             2968 non-null   object 
 15  4G             2968 non-null   object 
 16  Latitud        2968 non-null   float64
 17  Longitud       2968 non-null   float64
dtypes: float

,Provincia,Partido,Localidad,ADSL,CABLEMODEM,DIAL UP,FIBRA OPTICA,OTROS,SATELITAL,WIMAX,WIRELESS,Total general,Poblacion,TELEFONIAFIJA,3G,4G,Latitud,Longitud
0,BUENOS AIRES,25 de Mayo,25 de Mayo,914.0,4704.0,0.0,5.0,0.0,975.0,0.0,664.0,7262.0,23408.0,SI,SI,SI,-35.433939,-60.173121
1,BUENOS AIRES,25 de Mayo,Del Valle,181.0,0.0,0.0,0.0,10.0,1.0,0.0,0.0,192.0,899.0,--,SI,SI,-35.897345,-60.731609
2,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,0.0,0.0,0.0,0.0,0.0,0.0,0.0,181.0,181.0,547.0,--,--,--,-35.164397,-60.081358


* Cambio el orden de la columna *Población*:

In [39]:
column_order = ['Provincia', 'Partido', 'Localidad', 'Poblacion', 'ADSL', 'CABLEMODEM', 'FIBRA OPTICA', 'SATELITAL', 'WIMAX', 'WIRELESS', 'DIAL UP', 'OTROS', 'Total general', 'TELEFONIAFIJA', '3G', '4G', 'Latitud', 'Longitud']
internet_loc = internet_loc[column_order]

# Reviso:
internet_loc.head(2)

,Provincia,Partido,Localidad,Poblacion,ADSL,CABLEMODEM,FIBRA OPTICA,SATELITAL,WIMAX,WIRELESS,DIAL UP,OTROS,Total general,TELEFONIAFIJA,3G,4G,Latitud,Longitud
0,BUENOS AIRES,25 de Mayo,25 de Mayo,23408.0,914.0,4704.0,5.0,975.0,0.0,664.0,0.0,0.0,7262.0,SI,SI,SI,-35.433939,-60.173121
1,BUENOS AIRES,25 de Mayo,Del Valle,899.0,181.0,0.0,0.0,1.0,0.0,0.0,0.0,10.0,192.0,--,SI,SI,-35.897345,-60.731609


* Reviso los nulos del nuevo dataframe:

In [40]:
internet_loc.isnull().sum()

Provincia        0
Partido          0
Localidad        0
Poblacion        0
ADSL             0
CABLEMODEM       0
FIBRA OPTICA     0
SATELITAL        0
WIMAX            0
WIRELESS         0
DIAL UP          0
OTROS            0
Total general    0
TELEFONIAFIJA    0
3G               0
4G               0
Latitud          0
Longitud         0
dtype: int64

* Reviso duplicados:

In [41]:
internet_loc[internet_loc.duplicated(keep=False)==True]

,Provincia,Partido,Localidad,Poblacion,ADSL,CABLEMODEM,FIBRA OPTICA,SATELITAL,WIMAX,WIRELESS,DIAL UP,OTROS,Total general,TELEFONIAFIJA,3G,4G,Latitud,Longitud
1383,ENTRE RIOS,Paraná,El Ramblón,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,18.0,--,--,SI,-31.848047,-60.092749
1384,ENTRE RIOS,Paraná,El Ramblón,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,18.0,--,--,SI,-31.848047,-60.092749
2068,RIO NEGRO,General Roca,Barrio La Costa,94.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,--,--,--,-39.075307,-67.539226
2069,RIO NEGRO,General Roca,Barrio La Costa,94.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,--,--,--,-39.075307,-67.539226


* Elimino las filas duplicadas:

In [42]:
internet_loc.drop_duplicates(inplace=True)

# Reviso:
internet_loc[internet_loc.duplicated(keep=False)==True]

<ipython-input-42-ade25d6759ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  internet_loc.drop_duplicates(inplace=True)


,Provincia,Partido,Localidad,Poblacion,ADSL,CABLEMODEM,FIBRA OPTICA,SATELITAL,WIMAX,WIRELESS,DIAL UP,OTROS,Total general,TELEFONIAFIJA,3G,4G,Latitud,Longitud


## Continuaré con el último dataset relacionado al servicio de internet en Argentina:

* Ingresos_trimestrales_prestación_servicio_Internet_fijo

Guardo el dataset en un dataframe y especifico que los lea como *string* para que no haya error en la lectura de los datos, ya que contienen puntos.

In [43]:
internet_ingresos = pd.read_csv("/content/drive/MyDrive/HENRY/PI_DA/Datasets/Internet/Ingresos_trimestrales_prestación_servicio_Internet_fijo.csv", sep=",", dtype=str)

## **DF: internet_ingresos**

In [44]:
print(internet_ingresos.info())
internet_ingresos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Año                        33 non-null     object
 1   Trimestre                  33 non-null     object
 2   Ingresos (miles de pesos)  33 non-null     object
 3   Periodo                    33 non-null     object
dtypes: object(4)
memory usage: 1.2+ KB
None


,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2022,1,51.432.896,Ene-Mar 2022
1,2021,4,45.467.887,Oct-Dic 2021
2,2021,3,42.999.944,Jul-Sept 2021
3,2021,2,38.239.667,Abr-Jun 2021
4,2021,1,36.676.371,Ene-Mar 2021


* No hay nulos.
* La columna *Periodo* especifica los meses, pero esta infomación se deriva de la columna *Trimestre*, por lo que resulta redundante.
* El tipo de dato de las columnas *Año* y *Trimestre* deben ser cambiados a *integer*.
* Reemplazo los puntos por "" para luego poder convertirlo en float, y elimino la columna *Periodo*:

In [45]:
# Reemplazo los "." por "":
internet_ingresos["Ingresos (miles de pesos)"] = internet_ingresos["Ingresos (miles de pesos)"].apply(lambda x: x.replace('.', ''))

# Reemplazo el "-0" por "0":
#acc_tecnologia_prov[columnas] = acc_tecnologia_prov[columnas].apply(lambda x: x.str.replace('- 0', '0'))

# Convierto a float:
internet_ingresos["Ingresos (miles de pesos)"] = internet_ingresos["Ingresos (miles de pesos)"].astype(float)

# Convierto a integer:
internet_ingresos[["Año", "Trimestre"]]= internet_ingresos[["Año", "Trimestre"]].astype(int)

# Elimino la columna "Periodo":

internet_ingresos.drop("Periodo", axis=1, inplace= True)

# Reviso:
print(internet_ingresos.info())
internet_ingresos.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Año                        33 non-null     int64  
 1   Trimestre                  33 non-null     int64  
 2   Ingresos (miles de pesos)  33 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 920.0 bytes
None


,Año,Trimestre,Ingresos (miles de pesos)
0,2022,1,51432896.0
1,2021,4,45467887.0


# I.II DATASETS DE TELEFONÍA FIJA

Continuo con los datasets relacionados a los servicios de telefonía fija:
* Accesos_telefonía_fija_por_provincia-hogares-comercios-gobierno-otros
* Ingresos_trimestrales_telefonía fija.

Los importo a un dataframe  y especifico que los lea como *string* para que no haya error en la lectura de los datos, ya que contienen puntos:

In [46]:
telf_fija_acc_prov = pd.read_csv("/content/drive/MyDrive/HENRY/PI_DA/Datasets/Telefonia Fija/Accesos_telefonía_fija_por_provincia-hogares-comercios-gobierno-otros.csv", sep=",", dtype=str)
telf_fija_ingresos = pd.read_csv("/content/drive/MyDrive/HENRY/PI_DA/Datasets/Telefonia Fija/Ingresos_trimestrales_telefonía fija.csv", sep=",", dtype=str)

## **DF: telf_fija_acc_prov**

In [47]:
# Reviso:

print(telf_fija_acc_prov.info())
telf_fija_acc_prov.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 8 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Año                                 792 non-null    object
 1   Trimestre                           792 non-null    object
 2   Provincia                           792 non-null    object
 3   Accesos telefonía fija (total)      792 non-null    object
 4   Accesos telefonía fija (hogares)    792 non-null    object
 5   Accesos telefonía fija (comercial)  792 non-null    object
 6   Accesos telefonía fija (gobierno)   792 non-null    object
 7   Accesos telefonía fija (otros)      792 non-null    object
dtypes: object(8)
memory usage: 49.6+ KB
None


,Año,Trimestre,Provincia,Accesos telefonía fija (total),Accesos telefonía fija (hogares),Accesos telefonía fija (comercial),Accesos telefonía fija (gobierno),Accesos telefonía fija (otros)
0,2022,1,Buenos Aires,3.340.707,3.044.152,230.980,14.831,50.744
1,2022,1,Capital Federal,1.508.031,1.374.162,104.267,6.695,22.907
2,2022,1,Catamarca,30.167,27.489,2.086,134,458


* No hay nulos.
* Las columnas *Año* y *Trimestre* deben ser cambiadas a tipo *integer*.
* Se debe reemplazar los "." por "" en las columnas numéricas para después poder convertirlas a float.
* Se puede cambiar el orden de la columna *total* y colocarla al final para una mejor visualización.

In [48]:
# Defino las columnas:
columnas= ["Accesos telefonía fija (hogares)", "Accesos telefonía fija (comercial)",
           "Accesos telefonía fija (gobierno)", "Accesos telefonía fija (otros)", "Accesos telefonía fija (total)"]

# Reemplazo los "." por "":
telf_fija_acc_prov[columnas] = telf_fija_acc_prov[columnas].apply(lambda x: x.str.replace('.', ''))

# Convierto a float:
telf_fija_acc_prov[columnas] = telf_fija_acc_prov[columnas].astype(float)

# Convierto a integer:
telf_fija_acc_prov[["Año", "Trimestre"]]= telf_fija_acc_prov[["Año", "Trimestre"]].astype(int)

# Reordeno:
columnas_ord = ['Año', 'Trimestre', 'Provincia', 'Accesos telefonía fija (hogares)', 'Accesos telefonía fija (comercial)',
                'Accesos telefonía fija (gobierno)', 'Accesos telefonía fija (otros)', 'Accesos telefonía fija (total)']
telf_fija_acc_prov= telf_fija_acc_prov[columnas_ord]


# Reviso:
print(telf_fija_acc_prov.info())
telf_fija_acc_prov.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 8 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Año                                 792 non-null    int64  
 1   Trimestre                           792 non-null    int64  
 2   Provincia                           792 non-null    object 
 3   Accesos telefonía fija (hogares)    792 non-null    float64
 4   Accesos telefonía fija (comercial)  792 non-null    float64
 5   Accesos telefonía fija (gobierno)   792 non-null    float64
 6   Accesos telefonía fija (otros)      792 non-null    float64
 7   Accesos telefonía fija (total)      792 non-null    float64
dtypes: float64(5), int64(2), object(1)
memory usage: 49.6+ KB
None


<ipython-input-48-c11816f5fd98>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  telf_fija_acc_prov[columnas] = telf_fija_acc_prov[columnas].apply(lambda x: x.str.replace('.', ''))


,Año,Trimestre,Provincia,Accesos telefonía fija (hogares),Accesos telefonía fija (comercial),Accesos telefonía fija (gobierno),Accesos telefonía fija (otros),Accesos telefonía fija (total)
0,2022,1,Buenos Aires,3044152.0,230980.0,14831.0,50744.0,3340707.0
1,2022,1,Capital Federal,1374162.0,104267.0,6695.0,22907.0,1508031.0


* Ya que este dataset tiene por objetivo complementar información para el foco principal que es *cobertura de internet en Argentina*, no considero relevante mantener las columnas que especifican la cantidad de accesos para cada sector (hogar, comercial, etc) sino solo el total, ya que en los datasets sobre *internet* no proveen esta diferenciación.
* Por lo anterior , elimino las columnas innecesarias.

In [49]:
telf_fija_acc_prov.drop(["Accesos telefonía fija (hogares)", "Accesos telefonía fija (comercial)",
           "Accesos telefonía fija (gobierno)", "Accesos telefonía fija (otros)"], axis=1, inplace=True)

# Reviso:

telf_fija_acc_prov.head(2)

<ipython-input-49-1fc181ab097d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  telf_fija_acc_prov.drop(["Accesos telefonía fija (hogares)", "Accesos telefonía fija (comercial)",


,Año,Trimestre,Provincia,Accesos telefonía fija (total)
0,2022,1,Buenos Aires,3340707.0
1,2022,1,Capital Federal,1508031.0


## **DF: telf_fija_ingresos**

In [50]:
# Reviso:

print(telf_fija_ingresos.info())
telf_fija_ingresos.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Año                    33 non-null     object
 1   Trimestre              33 non-null     object
 2   Ingresos (miles de $)  33 non-null     object
dtypes: object(3)
memory usage: 920.0+ bytes
None


,Año,Trimestre,Ingresos (miles de $)
0,2014,1,"2.916.071,75"
1,2014,2,"2.974.865,2"
2,2014,3,"3.105.462,02"


* No hay nulos.
* Las columnas *Año* y *Trimestre* deben ser cambiadas a tipo *integer*.
* Se debe reemplazar los "." por "" en la columna *Ingresos (miles de $)* y las "," por "." , ya que corresponden a decimales,  para después poder convertirlas a float.


In [51]:
# Reemplazo los "." por "":
telf_fija_ingresos["Ingresos (miles de $)"] = telf_fija_ingresos["Ingresos (miles de $)"].apply(lambda x: x.replace('.', ''))

# Reemplazo las "," por ".":
telf_fija_ingresos["Ingresos (miles de $)"] = telf_fija_ingresos["Ingresos (miles de $)"].apply(lambda x: x.replace(',', '.'))

# Convierto a float:
telf_fija_ingresos["Ingresos (miles de $)"] = telf_fija_ingresos["Ingresos (miles de $)"].astype(float)

# Convierto a integer:
telf_fija_ingresos[["Año", "Trimestre"]]= telf_fija_ingresos[["Año", "Trimestre"]].astype(int)

# Reviso:
print(telf_fija_ingresos.info())
telf_fija_ingresos.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Año                    33 non-null     int64  
 1   Trimestre              33 non-null     int64  
 2   Ingresos (miles de $)  33 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 920.0 bytes
None


,Año,Trimestre,Ingresos (miles de $)
0,2014,1,2916071.75
1,2014,2,2974865.20


# I.III DATASETS DE TELEFONÍA MOVIL

Continuo con los datasets relacionados a los servicios de telefonía móvil:
* Ingresos_trimestrales_telefonía móvil
* Portabilidad_portaciones_netas_mensuales_por_operador
* Total_nacional_accesos_telefonia_movil-trimestral

Los importo a un dataframe  y especifico que los lea como *string* para que no haya error en la lectura de los datos, ya que contienen puntos:

In [52]:
telf_movil_ingresos = pd.read_csv("/content/drive/MyDrive/HENRY/PI_DA/Datasets/Telefonía Movil/Ingresos_trimestrales_telefonía móvil.csv", sep=",", dtype=str)
telf_movil_portab= pd.read_csv("/content/drive/MyDrive/HENRY/PI_DA/Datasets/Telefonía Movil/Portabilidad_portaciones_netas_mensuales_por_operador.csv", sep=",", dtype=str)
telf_movil_acc_total = pd.read_csv("/content/drive/MyDrive/HENRY/PI_DA/Datasets/Telefonía Movil/Total_nacional_accesos_telefonia_movil-trimestral.csv", sep=",", dtype=str)

## **DF: telf_movil_ingresos**

In [53]:
# Reviso:

print(telf_movil_ingresos.info())
telf_movil_ingresos.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Año                    37 non-null     object
 1   Trimestre              37 non-null     object
 2   Ingresos (miles de $)  37 non-null     object
 3   Periodo                37 non-null     object
dtypes: object(4)
memory usage: 1.3+ KB
None


,Año,Trimestre,Ingresos (miles de $),Periodo
0,2013,1,$ 13.135.235,Ene-Mar 2013
1,2013,2,$ 13.970.806,Abr-Jun 2013
2,2013,3,$ 15.235.659,Jul-Sept 2013


* No hay nulos.
* Las columnas *Año* y *Trimestre* deben ser cambiadas a integer.
* La columna *Periodo* especifica los meses , pero se desprende de la columna  *Trimestre* por lo que es irrelevante y puede ser eliminada.
* Se debe reemplazar los "." por "" en la columna *Ingresos (miles de $)*, así como eliminar el símbolo dólar, para después poder convertirlas a float.


In [54]:
# Reemplazo los "." por "":
telf_movil_ingresos["Ingresos (miles de $)"] = telf_movil_ingresos["Ingresos (miles de $)"].apply(lambda x: x.replace('.', ''))

# Reemplazo las "$" por "":
telf_movil_ingresos["Ingresos (miles de $)"] = telf_movil_ingresos["Ingresos (miles de $)"].apply(lambda x: x.replace('$ ', ''))

# Convierto a float:
telf_movil_ingresos["Ingresos (miles de $)"] = telf_movil_ingresos["Ingresos (miles de $)"].astype(float)

# Convierto a integer:
telf_movil_ingresos[["Año", "Trimestre"]]= telf_movil_ingresos[["Año", "Trimestre"]].astype(int)

# Elimino la columna "Periodo":
telf_movil_ingresos.drop("Periodo", axis=1, inplace=True)

# Reviso:
print(telf_movil_ingresos.info())
telf_movil_ingresos.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Año                    37 non-null     int64  
 1   Trimestre              37 non-null     int64  
 2   Ingresos (miles de $)  37 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 1016.0 bytes
None


,Año,Trimestre,Ingresos (miles de $)
0,2013,1,13135235.0
1,2013,2,13970806.0


## **DF: telf_movil_portab**

In [55]:
# Reviso:

print(telf_movil_portab.info())
telf_movil_portab.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Año       120 non-null    object
 1   Mes       120 non-null    object
 2   Personal  120 non-null    object
 3   Claro     120 non-null    object
 4   Nextel    70 non-null     object
 5   Movistar  120 non-null    object
 6   Periodo   120 non-null    object
dtypes: object(7)
memory usage: 6.7+ KB
None


,Año,Mes,Personal,Claro,Nextel,Movistar,Periodo
0,2012,3,-17,19,0,-2,Mar-12
1,2012,4,3.508,-1.134,-189,-2.185,Apr-12
2,2012,5,1.253,4.792,-853,-5.192,May-12


* Hay nulos en el campo *Nextel*.
* Las columnas *Año* y *Mes* deben ser cambiadas a tipo integer.
* La columna *Periodo* especifica el mes y año , pero se desprende de la columnas  *Año* y *Mes*, por lo que es irrelevante y puede ser eliminada.
* Se debe reemplazar los "." por "" en la columnas numéricas para después poder convertirlas a float.

In [56]:
# Defino columnas:
columnas=["Personal", "Claro", "Nextel", "Movistar"]

# Reemplazo los "." por "":
telf_movil_portab[columnas] = telf_movil_portab[columnas].apply(lambda x: x.str.replace('.', ''))

# Convierto a float:
telf_movil_portab[columnas] = telf_movil_portab[columnas].astype(float)

# Convierto a integer:
telf_movil_portab[["Año", "Mes"]]= telf_movil_portab[["Año", "Mes"]].astype(int)

# Elimino la columna "Periodo":
telf_movil_portab.drop("Periodo", axis=1, inplace=True)

# Reviso:
print(telf_movil_portab.info())
telf_movil_portab.head(2)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Año       120 non-null    int64  
 1   Mes       120 non-null    int64  
 2   Personal  120 non-null    float64
 3   Claro     120 non-null    float64
 4   Nextel    70 non-null     float64
 5   Movistar  120 non-null    float64
dtypes: float64(4), int64(2)
memory usage: 5.8 KB
None


<ipython-input-56-d34c27bf5164>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  telf_movil_portab[columnas] = telf_movil_portab[columnas].apply(lambda x: x.str.replace('.', ''))


,Año,Mes,Personal,Claro,Nextel,Movistar
0,2012,3,-17.0,19.0,0.0,-2.0
1,2012,4,3508.0,-1134.0,-189.0,-2185.0


* Reviso los nulos:

In [57]:
telf_movil_portab["Nextel"].isnull().sum()

50

* Hay 50 nulos en el campo *Nextel*; sin embargo, esta marca no provee en Argentina servicio de internet fijo, por lo que no es una variable de interés para el análisis.
* Elimino la columna *Nextel*:

In [58]:
telf_movil_portab.drop("Nextel", axis=1, inplace=True)

#Reviso:
telf_movil_portab.head(2)

,Año,Mes,Personal,Claro,Movistar
0,2012,3,-17.0,19.0,-2.0
1,2012,4,3508.0,-1134.0,-2185.0


## **DF: telf_movil_acc_total**

In [59]:
print(telf_movil_acc_total.info())
telf_movil_acc_total.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Año                          37 non-null     object
 1   Trimestre                    37 non-null     object
 2   Total de accesos pospago     37 non-null     object
 3   Total de accesos prepago     37 non-null     object
 4   Total de accesos operativos  37 non-null     object
 5   Periodo                      37 non-null     object
dtypes: object(6)
memory usage: 1.9+ KB
None


,Año,Trimestre,Total de accesos pospago,Total de accesos prepago,Total de accesos operativos,Periodo
0,2013,1,7.761.349,57.147.608,64.908.957,Ene-Mar 2013
1,2013,2,7.769.110,58.004.822,65.773.932,Abr-Jun 2013
2,2013,3,8.002.183,59.048.909,67.051.092,Jul-Sept 2013


* No hay nulos.
* Las columnas *Año* y *Trimestre* deben ser cambiadas a tipo integer.
* La columna *Periodo* especifica los meses y años , pero se desprende de la columnas  *Año* y *Trimestre*, por lo que es irrelevante y puede ser eliminada.
* Se debe reemplazar los "." por "" en la columnas numéricas para después poder convertirlas a float.

In [60]:
# Defino columnas:
columnas=["Total de accesos pospago", "Total de accesos prepago", "Total de accesos operativos"]

# Reemplazo los "." por "":
telf_movil_acc_total[columnas] = telf_movil_acc_total[columnas].apply(lambda x: x.str.replace('.', ''))

# Convierto a float:
telf_movil_acc_total[columnas] = telf_movil_acc_total[columnas].astype(float)

# Convierto a integer:
telf_movil_acc_total[["Año", "Trimestre"]]= telf_movil_acc_total[["Año", "Trimestre"]].astype(int)

# Elimino la columna "Periodo":
telf_movil_acc_total.drop("Periodo", axis=1, inplace=True)

# Reviso:
print(telf_movil_acc_total.info())
telf_movil_acc_total.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Año                          37 non-null     int64  
 1   Trimestre                    37 non-null     int64  
 2   Total de accesos pospago     37 non-null     float64
 3   Total de accesos prepago     37 non-null     float64
 4   Total de accesos operativos  37 non-null     float64
dtypes: float64(3), int64(2)
memory usage: 1.6 KB
None


<ipython-input-60-0b94e1675321>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  telf_movil_acc_total[columnas] = telf_movil_acc_total[columnas].apply(lambda x: x.str.replace('.', ''))


,Año,Trimestre,Total de accesos pospago,Total de accesos prepago,Total de accesos operativos
0,2013,1,7761349.0,57147608.0,64908957.0
1,2013,2,7769110.0,58004822.0,65773932.0


**¡LISTO!** Terminé con las transformaciones necesarias.

# **III. CARGA**

Finalmente, exporto los dataframes resultantes a nuevos csv´s, a partir de los cuales realizaré el EDA.

In [61]:
# Internet
acc_velocidad_rangos.to_csv('/content/drive/MyDrive/HENRY/PI_DA/POST_ETL_Datasets/internet_rangos_vel_prov.csv', index=False)
internet_prov.to_csv('/content/drive/MyDrive/HENRY/PI_DA/POST_ETL_Datasets/internet_prov.csv', index=False)
internet_loc.to_csv('/content/drive/MyDrive/HENRY/PI_DA/POST_ETL_Datasets/internet_loc.csv', index=False)
internet_ingresos.to_csv('/content/drive/MyDrive/HENRY/PI_DA/POST_ETL_Datasets/internet_ingresos.csv', index=False)

# Telefonía Fija
telf_fija_ingresos.to_csv("/content/drive/MyDrive/HENRY/PI_DA/POST_ETL_Datasets/telf_fija_ingresos.csv", index=False)
telf_fija_acc_prov.to_csv("/content/drive/MyDrive/HENRY/PI_DA/POST_ETL_Datasets/telf_fija_acc_prov.csv", index=False)

# Telefonía móvil
telf_movil_acc_total.to_csv("/content/drive/MyDrive/HENRY/PI_DA/POST_ETL_Datasets/telf_movil_acc_total.csv", index=False)
telf_movil_portab.to_csv("/content/drive/MyDrive/HENRY/PI_DA/POST_ETL_Datasets/telf_movil_portab.csv", index=False)
telf_movil_ingresos.to_csv("/content/drive/MyDrive/HENRY/PI_DA/POST_ETL_Datasets/telf_movil_ingresos.csv", index=False)

